In [1]:
import pandas as pd

In [2]:
src_table = pd.read_excel("DataSources.xlsx",index_col=0)

In [7]:
sources = list(set(src_table.SRC.to_list()))

In [8]:
sources

['GDP', 'FPI', 'NG', 'FERT', 'USDEUR']

In [10]:
src_table[src_table.SRC==sources[0]]


,API,SRC,TYPE,Download type,STATUS,S3 Location
5,FRED,GDP,A,US GDP - Historical,Complete,s3://insights-canpotex-downloads-sandboxlz/Fre...
6,EIA,GDP,F,US GDP - Forecast STEO,Complete,s3://insights-canpotex-downloads-sandboxlz/EIA...


# EURUSD

In [5]:
url = 's3://insights-canpotex-downloads-sandboxlz/Fred-downloads/EXUSEU/EXUSEU.csv'
df = pd.read_csv(url,index_col=0,parse_dates=True)
df # REQUIRES UPDATE!!!!!

,rate
date,
2014-01-01,1.3618
2014-02-01,1.3665
2014-03-01,1.3828
2014-04-01,1.3810
2014-05-01,1.3739
...,...
2022-05-01,1.0567
2022-06-01,1.0567
2022-07-01,1.0168


# EURUSD - Forecast

In [3]:
url = 's3://insights-canpotex-downloads-sandboxlz/OECD-downloads/EXCHUD/EXCHUD.csv'
df = pd.read_csv(url,index_col=0,parse_dates=True)
df # Time update and resampling required 

,rate
date,
2014-01-01,0.753695
2015-01-01,0.901452
2016-01-01,0.904282
2017-01-01,0.887107
2018-01-01,0.847324
2019-01-01,0.893348
2020-01-01,0.877542
2021-01-01,0.845553
2022-01-01,0.931910


#  US GDP

In [11]:
url = 's3://insights-canpotex-downloads-sandboxlz/Fred-downloads/US_GDP/US_GDP.csv'
df = pd.read_csv(url,index_col=0,parse_dates=True)
df #Requires update | Resample to monthly | Index as datetime

,US_GDP
date,
2014-01-01,17144.281
2014-04-01,17462.703
2014-07-01,17743.227
2014-10-01,17852.540
2015-01-01,17991.348
2015-04-01,18193.707
2015-07-01,18306.960
2015-10-01,18332.079
2016-01-01,18425.306


In [ ]:
actual | forecast | consolidated

# US GDP - forecast

In [ ]:
url = 's3://insights-canpotex-downloads-sandboxlz/EIA-downloads/HenryHubGasPrice/nominalGDP.csv'
df = pd.read_csv(url,index_col=0,parse_dates=True)
df # Resample to monthly | Index as Datetime | Why do we have Qtr pricing (historical)

# NATURAL GAS PRICES

In [12]:
url = 's3://insights-canpotex-downloads-sandboxlz/EIA-downloads/HenryHubGasPrice/gasprice.csv'
df = pd.read_csv(url,index_col=0,parse_dates=True)
df #Requires update | Resample to monthly | Index as datetime

,realGDP,deflator,nominalGDP
date,,,
2014 Q1,16654.247000,102.965000,1.714805e+06
2014 Q2,16868.109000,103.552000,1.746726e+06
2014 Q3,17064.616000,104.029000,1.775215e+06
2014 Q4,17141.235000,104.104000,1.784471e+06
2015 Q1,17280.647000,104.092000,1.798777e+06
2015 Q2,17380.875000,104.683000,1.819482e+06
2015 Q3,17437.080000,104.939000,1.829830e+06
2015 Q4,17462.579000,104.932000,1.832383e+06
2016 Q1,17565.465000,104.873000,1.842143e+06


# NATURAL GAS FORECAST

In [16]:
url = 's3://insights-canpotex-downloads-sandboxlz/EIA-downloads/HenryHubGasPrice/gaspricesteo.csv'
df = pd.read_csv(url,index_col=0,parse_dates=True)
df # (need a tag for last updated) | Why do we have quarterly pricing above?

,"Natural Gas Henry Hub Spot Price ($/mmBtu), Monthly (dollars per million Btu)"
date,
2014-01-01,4.713000
2014-02-01,5.999000
2014-03-01,4.903000
2014-04-01,4.658000
2014-05-01,4.581000
...,...
2023-08-01,5.311649
2023-09-01,5.227389
2023-10-01,5.252002


# TOTAL FERTILIZER PRODUCTION

In [22]:
url = 's3://insights-canpotex-downloads-sandboxlz/FAO-downloads/totalFertilizationProductionHist/totalFertilizerProduction.csv'
df = pd.read_csv(url,index_col=0,parse_dates=True)
df # No dates...

,Production
0,8.077018e+07
1,8.557887e+07
2,9.627682e+07
3,1.071527e+08
4,1.207713e+08
5,1.349903e+08
6,1.478641e+08
7,1.574102e+08
8,1.665242e+08
9,1.792081e+08


# TOTAL FERTILIZER PRODUCTION FORECAST

In [23]:
url = None
df = pd.read_csv(url,index_col=0,parse_dates=True)
df # (need a tag for last updated) | Why do we have quarterly pricing above?

ValueError: Invalid file path or buffer object type: <class 'NoneType'>

# FOOD PRICE INDEX

In [26]:
url = 's3://insights-canpotex-downloads-sandboxlz/FAO-downloads/FoodPriceIndex/FoodPriceIndex-hist.csv'
df = pd.read_csv(url,index_col=0,parse_dates=True)
df #Requires cleanup NaN values and index as datetime

,FoodPriceIndex
Date,
1990-01-01,64.1
1990-02-01,64.5
1990-03-01,63.8
1990-04-01,65.8
1990-05-01,64.4
...,...
NaT,NaN
NaT,NaN
NaT,NaN


# FOOD PRIE INDEX FORECAST

In [27]:
url = 's3://insights-canpotex-downloads-sandboxlz/FAO-downloads/FoodPriceIndex/FoodPriceIndex-forecast.csv'
df = pd.read_csv(url,index_col=0,parse_dates=True)
df #Requires cleanup NaN values and index as datetime

,FoodPriceIndex
year,
1991-01-01,62.100000
1992-01-01,63.900000
1993-01-01,61.900000
1994-01-01,66.900000
1995-01-01,76.600000
1996-01-01,77.600000
1997-01-01,70.600000
1998-01-01,64.700000
1999-01-01,55.200000
